In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import yfinance as yf

c:\Users\Lenovo\OneDrive\my code\AI&ML\LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
def get_stock_data(ticker):
    data = yf.download(
        ticker,
        period="5y",
        interval="1d",
        auto_adjust=True
    )

    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [col[0] for col in data.columns]

    required_cols = ["Open", "High", "Low", "Close"]
    missing_cols = [col for col in required_cols if col not in data.columns]
    if data.empty or missing_cols:
        return None

    data = data.dropna(subset=required_cols)
    data.reset_index(inplace=True)
    return data

In [20]:
data=get_stock_data('AAPL')
data

[*********************100%***********************]  1 of 1 completed


,Date,Close,High,Low,Open,Volume
0,2020-07-09,93.078072,93.627286,92.028235,93.573822,125642800
1,2020-07-10,93.240868,93.299197,92.059807,92.672208,90257200
2,2020-07-13,92.810738,97.163178,92.596881,94.548310,191649200
3,2020-07-14,94.346603,94.538582,91.255423,92.191034,170989200
4,2020-07-15,94.995453,96.475427,93.794947,96.225120,153198000
...,...,...,...,...,...,...
1250,2025-07-01,207.820007,210.190002,206.139999,206.669998,78788900
1251,2025-07-02,212.440002,213.339996,208.139999,208.910004,67941800
1252,2025-07-03,213.550003,214.649994,211.809998,212.149994,34955800
1253,2025-07-07,209.949997,216.229996,208.800003,212.679993,50229000


In [21]:
df_prophet = data[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})


In [31]:
df_prophet.shape

(1255, 2)

In [22]:
train = df_prophet[:-200]
test = df_prophet[-200:]


In [23]:
train.shape


(1055, 2)

In [24]:
test.shape

(200, 2)

In [25]:
model = Prophet(daily_seasonality=True, yearly_seasonality=True)
model.fit(train)


12:10:09 - cmdstanpy - INFO - Chain [1] start processing
12:10:10 - cmdstanpy - INFO - Chain [1] done processing


In [26]:
future = model.make_future_dataframe(periods=60)
forecast = model.predict(future)

In [27]:
forecast_test = forecast[['ds', 'yhat']].set_index('ds').join(test.set_index('ds'))
forecast_test.dropna(inplace=True)

In [28]:
forecast_test

,yhat,y
ds,,
2024-09-18,217.369425,219.916977
2024-09-19,216.886498,228.068314
2024-09-20,216.762433,227.400650
2024-09-23,216.466073,225.676727
2024-09-24,216.192080,226.573547
2024-09-25,216.089084,225.577057
2024-09-26,215.902663,226.723038
2024-09-27,216.068427,226.992081
2024-09-30,216.558187,232.183853


In [29]:
forecast_test = forecast_test.rename(columns={'y': 'Close'})
forecast_test.reset_index(inplace=True)
forecast_test

,ds,yhat,Close
0,2024-09-18,217.369425,219.916977
1,2024-09-19,216.886498,228.068314
2,2024-09-20,216.762433,227.400650
3,2024-09-23,216.466073,225.676727
4,2024-09-24,216.192080,226.573547
5,2024-09-25,216.089084,225.577057
6,2024-09-26,215.902663,226.723038
7,2024-09-27,216.068427,226.992081
8,2024-09-30,216.558187,232.183853
9,2024-10-01,216.506530,225.417633


In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

forecast_test['Close'] = pd.to_numeric(forecast_test['Close'], errors='coerce')
forecast_test['yhat'] = pd.to_numeric(forecast_test['yhat'], errors='coerce')

forecast_test.dropna(subset=['Close', 'yhat'], inplace=True)

mae = mean_absolute_error(forecast_test['Close'], forecast_test['yhat'])
rmse = np.sqrt(mean_squared_error(forecast_test['Close'], forecast_test['yhat']))
mape = (abs((forecast_test['Close'] - forecast_test['yhat']) / forecast_test['Close'])).mean() * 100

print(f"📉 MAE: {mae:.2f}")
print(f"📉 RMSE: {rmse:.2f}")
print(f"📉 MAPE: {mape:.2f}%")


📉 MAE: 8.26
📉 RMSE: 9.59
📉 MAPE: 3.60%


In [32]:
forecast_test

,ds,yhat,Close
0,2024-09-18,217.369425,219.916977
1,2024-09-19,216.886498,228.068314
2,2024-09-20,216.762433,227.400650
3,2024-09-23,216.466073,225.676727
4,2024-09-24,216.192080,226.573547
5,2024-09-25,216.089084,225.577057
6,2024-09-26,215.902663,226.723038
7,2024-09-27,216.068427,226.992081
8,2024-09-30,216.558187,232.183853
9,2024-10-01,216.506530,225.417633


In [33]:
print("Average Close:", forecast_test['Close'].mean())
print("Correct MAPE:", (abs((forecast_test['Close'] - forecast_test['yhat']) / forecast_test['Close'])).mean() * 100)


Average Close: 227.37260224098384
Correct MAPE: 3.6033606540533802
